## Calculating the size of the genome

In [3]:
# open an
with open("../data/row_sequence.txt", "r") as sequence: 
    row= sequence.read().replace('\n', '').replace('\r', '').replace('\t', '')
genome_size = len(row)
print("The size of the genome is {0} bp".format(genome_size))

The size of the genome is 10676 bp


## Generating a fasta file

In [6]:
fasta = ">my_sequence|2023|Monastir\n"+row+"\n"
with open("unk.fa", "w") as fasta_file: 
    fasta_file.write(fasta)

## Performing a BLAST analysis programmarly
The below code will test if a connexion to NCBI server could be established

In [9]:
from Bio import Entrez

# Set the email address for your NCBI account
Entrez.email = "houcemoo@gmail.com"

# Test the connection to the NCBI server
try:
    handle = Entrez.einfo()
    print("Connection to NCBI server successful")
    handle.close()
except Exception as e:
    print("Connection to NCBI server failed:", e)

Connection to NCBI server successful


Now we can query the generated sequence

In [11]:
from Bio.Blast import NCBIWWW

result_handle = NCBIWWW.qblast("blastn", "nt", row)

with open("blast_results.xml", "w") as output_file:
    output_file.write(result_handle.read())

result_handle.close()


## Counting the number of complete Zika virus genomes in GenBank

In [20]:
# Define the search term
search_term = '"Zika virus"[ORGANISM] AND "complete genome"[TITLE]'

# Perform the search
handle = Entrez.esearch(db="nucleotide", term=search_term, retmax=10000)
record = Entrez.read(handle)

# Get the count of entries
count = record["Count"]
gis = record["IdList"]

print("Number of entries for Zika virus sequences in GenBank:", count)
print("These are the first 10 accession numbers:", gis[0:10] )


Number of entries for Zika virus sequences in GenBank: 336
These are the first 10 accession numbers: ['2471615310', '2471615308', '2471615306', '2471615304', '2471615302', '2471615300', '2471615298', '2471615296', '1696858399', '2411462333']


## We can download the FASTA files using the list of sequence IDs

In [21]:
from Bio import SeqIO

# Define the list of GI codes
gi_codes = ["2471615310", "2471615308", "2471615306"]  

# Retrieve the FASTA sequences
handle = Entrez.efetch(db="nucleotide", id=gi_codes, rettype="fasta", retmode="text")
records = SeqIO.parse(handle, "fasta")

# Save each sequence as a separate FASTA file
for record in records:
    filename = f"{record.id}.fasta"
    SeqIO.write(record, filename, "fasta")
    print(f"FASTA sequence saved: {filename}")

handle.close()


FASTA sequence saved: OQ661919.1.fasta
FASTA sequence saved: OQ661918.1.fasta
FASTA sequence saved: OQ661917.1.fasta


## Fetching a GenBank file

In [22]:
# Accession number of the GenBank entry
accession_number = "2471615310"  # Replace with the actual accession number

# Fetch the GenBank record
handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="gb", retmode="text")
genbank_record = handle.read()

# Save the GenBank record to a file
output_file = "2471615310.gb"  # Replace with the desired output file path
with open(output_file, "w") as file:
    file.write(genbank_record)

# Close the handle
handle.close()

print("GenBank record fetched and saved successfully.")


GenBank record fetched and saved successfully.


## Parsing GB file to extract useful files

In [25]:
genbank_file = "2471615310.gb"  # Replace with the path to your GenBank file

# Parse the GenBank file and extract desired information
for record in SeqIO.parse(genbank_file, "genbank"):
    accession = record.id
    country = record.features[0].qualifiers.get("country", ["Unknown"])[0]
    collection_date = record.features[0].qualifiers.get("collection_date", ["Unknown"])[0]
    host = record.features[0].qualifiers.get("host", ["Unknown"])[0]

    print(f"Accession: {accession}")
    print(f"Country: {country}")
    print(f"Collection Date: {collection_date}")
    print(f"Host: {host}")
    print("-----------------------")


Accession: OQ661919.1
Country: Thailand
Collection Date: 06-Nov-2018
Host: Homo sapiens
-----------------------
